In [34]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [35]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

a = np.array([[1,2,3,8,5], [12, 23, 12, 9, 43], [0, 23, 8, 8, 4]])
a_3 = a[:, 3]
a_without_3 = a[:, np.array(range(a.shape[1])) != 3]

a_sep = []
for i in np.unique(a[:, 3]):
    a_sep.append(a_without_3[a_3 == i, :])
a_sep

In [39]:
feature_21 = x[:, 21]
x_minus_21 = x[:, np.array(range(x.shape[1])) != 21]
x_sep = []
for i in np.unique(feature_21):
    x_sep.append(x_minus_21[feature_21 == i, :])

for i, x_cat in enumerate(x_sep):
    x_sep[i] = sanitize(x_cat)
    
y = y.reshape((y.shape[0], 1))

In [3]:
sanitized_y = y.reshape((y.shape[0], 1))

In [ ]:
for i, x_cat in enumerate(x_sep):
    x_sep[i] = sanitize(x_cat)

In [40]:
w_init = np.random.rand(sanitized_x.shape[1], 1)
print(w_init.shape)
w, loss = least_squares_GD(sanitized_y, sanitized_x, np.random.rand(sanitized_x.shape[1], 1), 1000, 0.01, verbose=True)
print(w, loss)

(30, 1)


TypeError: print_info() missing 1 required positional argument: 'N'

In [31]:
prediction = sanitized_x @ w
print(prediction)

prediction[prediction >= 0] = 1
prediction[prediction <= 0] = -1

print(prediction)

print(np.sum(prediction == sanitized_y)/sanitized_y.shape[0])

[[-0.25516513]
 [ 0.31391931]
 [-0.42809016]
 ...
 [-0.0349503 ]
 [ 0.41117694]
 [-0.33713466]]
[[-1.]
 [ 1.]
 [-1.]
 ...
 [-1.]
 [ 1.]
 [-1.]]
0.71338


In [ ]:
print(compute_accuracy(sanitized_y, sanitized_x, ))

Reduce dimensions and standardize the columns of X

Running logistic regression with different values of degrees (polynomial feature expansion), gamma and different initial weights

In [6]:
print(y_train.shape, tx_train.shape, w.shape)

NameError: name 'w' is not defined

In [9]:
N = sanitized_x.shape[0]
inds = np.random.choice(range(N), 100000, replace=False)
sanitized_x_sub = sanitized_x[inds, :]
sanitized_x_sub = sanitized_x_sub[:, [0,1,2,3,7,8,9,10,11,13,14,17,19,21,22,29]]
sanitized_y_sub = sanitized_y[inds]
x_train, x_test, y_train, y_test = split_data(sanitized_x, sanitized_y, 0.8)
number_of_w = 1

for degree in range(1, 5)
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    for gamma in np.array([0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 1000, gamma, verbose=True)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

Running logistic regression with polynomial feature expansion with degree 1 and gamma 0.01 , with 1 different initial weights.
grad norm: 199306.688498 with shape (31, 1)
Iteration : 0 with loss -145659968.18611288 and gradient norm: 0.996533442488351
grad norm: 107193.921865 with shape (31, 1)
Iteration : 250 with loss -31061156022.882534 and gradient norm: 0.5359696093259502
grad norm: 107193.918952 with shape (31, 1)
Iteration : 500 with loss -61890059878.312744 and gradient norm: 0.5359695947615115
grad norm: 107193.91872 with shape (31, 1)
Iteration : 750 with loss -92719347883.98398 and gradient norm: 0.5359695935982532
(200000, 1) (200000, 31) (31, 1)
0 :
Accuracy on training set 0.70926
Accuracy on test set:,  0.70512
Running logistic regression with polynomial feature expansion with degree 1 and gamma 0.05 , with 1 different initial weights.
grad norm: 202614.604895 with shape (31, 1)
Iteration : 0 with loss -816451996.3234181 and gradient norm: 1.0130730244748851
grad norm: 1

In [15]:
np.unique(x[:, 22])

array([ 0.,  1.,  2.,  3.])

In [ ]:
N = sanitized_x.shape[0]
inds = np.random.choice(range(N), 100000, replace=False)
sanitized_x_sub = sanitized_x[inds, :]
sanitized_x_sub = sanitized_x_sub[:, [0,1,2,3,7,8,9,10,11,13,14,17,19,21,22,29]]
sanitized_y_sub = sanitized_y[inds]
x_train, x_test, y_train, y_test = split_data(sanitized_x, sanitized_y, 0.8)
number_of_w = 1

sanitized_x[:, 22]

for degree in range(1, 5)
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    for gamma in np.array([0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 1000, gamma, verbose=True)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

## Generate predictions and save ouput in csv format for submission:


In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [346]:
a = np.random.rand(30)
np.around(a)

array([1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [347]:
a

array([0.98948792, 0.67934654, 0.34673991, 0.48885077, 0.5348651 ,
       0.08436905, 0.02071519, 0.18776387, 0.52188862, 0.85832073,
       0.50843456, 0.60499752, 0.84728409, 0.93295583, 0.00486189,
       0.66121463, 0.3157777 , 0.6534353 , 0.12888288, 0.54033186,
       0.44024959, 0.37952039, 0.0833081 , 0.55721317, 0.36309742,
       0.2762448 , 0.46041519, 0.00914704, 0.35695435, 0.45490402])